In [1]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
import django
django.setup()

In [2]:
from django.db import connection
connection.connect()

In [3]:
from fotoweb.models import *

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options

In [11]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service

class SelDriver():
  def __init__(self,dbg=False):
    chrome_options = Options()
    #chrome_options.add_argument("--headless")
    if dbg:
        chrome_options.add_experimental_option("debuggerAddress", "127.0.0.1:9222")
    #Change chrome driver path accordingly
    #chrome_driver = "chromedriver.exe
    #chrome_driver = r"c:\programs\chrome-win64"
    service = Service()
    self.driver = webdriver.Chrome(service=service, options=chrome_options)
    print (self.driver)
    self.driver.implicitly_wait(5)
    self.vars = {}

In [12]:
driver=SelDriver(True)

<selenium.webdriver.chrome.webdriver.WebDriver (session="50927f699140ba075379b4440c52cc6e")>


In [ ]:

#ee=driver.driver.find_elements(By.CSS_SELECTOR, '.cm-media-thumb img')
ee=driver.driver.find_elements(By.CSS_SELECTOR, '.cm-media-thumb img')
ee

In [ ]:
srcs=[]
for e in ee:
    src=e.get_attribute('src')
    r=re.search(r'(\d+)\.jpg',src)
    print(src,r.group(1))
    srcs.append(r.group(1))



In [ ]:
images=[]
for src in srcs:
    driver.driver.get('https://submit.shutterstock.com/catalog_manager/images/'+src)
    name = driver.driver.find_element(By.CSS_SELECTOR,'.o_List_List_itemText div').text
    description = driver.driver.find_element(By.CSS_SELECTOR,'.o_EditorDescription_EditorDescription_descriptionInput').text
    images.append({
        'name':name,
        'description':description,
        'shutter-id':src
    })
    i=None
    try:
        i=Image.objects.get(name=name)
        print(name,'found exact match')
    except:
        name = driver.driver.find_element(By.CSS_SELECTOR,'.o_List_List_itemText div').text.replace('.jpg','')
        try:
            i=Image.objects.filter(name__contains=name).first()
            print(name,'found by contains')
        except:
            print(name,'NOT found')

    if i and i.shutter:
        print('already shutter')
    elif i:
        i.shutter_dt=i.updated_dt
        if not i.title:
            i.title=driver.driver.find_element(By.CSS_SELECTOR,'.o_EditorDescription_EditorDescription_descriptionInput').text
        i.shutter=True
        i.shutter_url='https://submit.shutterstock.com/catalog_manager/images/'+src
        i.save()
        print('shutter updated')


In [ ]:
images

In [26]:
driver.driver.get("https://submit.shutterstock.com/en/data_catalog")

In [ ]:
cards = driver.driver.find_elements(By.CSS_SELECTOR, ".MuiGrid-root > .MuiBox-root > .MuiPaper-root")
cards

In [43]:
for c in cards:
    #print(c.text)
    idiv = c.find_element(By.CSS_SELECTOR,".MuiBox-root:nth-child(1) > .MuiTypography-root")
    name = idiv.text
    print(name)

    i=None
    try:
        i=Image.objects.get(name=name)
        print(name,'found exact match')
    except:
        print(name,'NOT found')

    if i and i.shutter:
        print('already shutter')
    elif i:
        i.shutter_dt=i.updated_dt
        i.shutter=True
        i.shutter_url='Data licensing'
        i.save()
        print('shutter updated')

                       

23-08-31-DSC03818-1.jpg
23-08-31-DSC03818-1.jpg found exact match


shutter updated
23-08-31-DSC03855-1.jpg
23-08-31-DSC03855-1.jpg found exact match
shutter updated
23-09-22-DSC04051-1.jpg
23-09-22-DSC04051-1.jpg found exact match
shutter updated
23-08-31-DSC03807-1.jpg
23-08-31-DSC03807-1.jpg found exact match
shutter updated
23-08-31-DSC03814-1.jpg
23-08-31-DSC03814-1.jpg found exact match
shutter updated
23-08-31-DSC03778-1.jpg
23-08-31-DSC03778-1.jpg found exact match
shutter updated
23-08-31-DSC03788-1.jpg
23-08-31-DSC03788-1.jpg found exact match
shutter updated
23-08-31-DSC03797-1.jpg
23-08-31-DSC03797-1.jpg found exact match
shutter updated
23-08-31-DSC03779-1.jpg
23-08-31-DSC03779-1.jpg found exact match
shutter updated
23-08-31-DSC03773-1.jpg
23-08-31-DSC03773-1.jpg found exact match
shutter updated
23-03-03-DJI_0036-1.jpg
23-03-03-DJI_0036-1.jpg found exact match
shutter updated
23-03-13-DSC09971-1.jpg
23-03-13-DSC09971-1.jpg found exact match
shutter updated
23-03-13-DSC09976-1.jpg
23-03-13-DSC09976-1.jpg found exact match
shutter updated
